In [59]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [60]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [61]:
tf.reset_default_graph()
# Training Params
num_steps = 70000
batch_size = 128
learning_rate = 0.0002

# Network Params
image_dim = 784 # 28*28 pixels
hidden_gen = 256
hidden_dis = 256
noise_dim = 100 # Noise data points


In [62]:
weights = {
    'gen_w1': tf.get_variable('gen_w1', shape=[noise_dim, hidden_gen], dtype=tf.float32, 
                             initializer = tf.random_normal_initializer(stddev=1. / tf.sqrt(noise_dim/ 2.))),
    'gen_w_out': tf.get_variable('gen_w_out', shape=[ hidden_gen, image_dim], dtype=tf.float32, 
                             initializer = tf.random_normal_initializer(stddev=1. / tf.sqrt(hidden_gen / 2.))),
    'dis_w1' : tf.get_variable('dis_w1', shape=[image_dim, hidden_dis], dtype=tf.float32, 
                             initializer = tf.random_normal_initializer(stddev=1. / tf.sqrt(image_dim/ 2.))),
    'dis_w_out' : tf.get_variable('dis_w_out', shape=[ hidden_dis, 1], dtype=tf.float32, 
                             initializer = tf.random_normal_initializer(stddev=1. / tf.sqrt( hidden_dis / 2.)))
}

biases = {
    'gen_b1': tf.get_variable('gen_b1', shape=[hidden_gen], dtype=tf.float32, 
                             initializer = tf.zeros_initializer()),
    'gen_b_out': tf.get_variable('gen_b_out', shape=[image_dim], dtype=tf.float32, 
                             initializer = tf.zeros_initializer()),
    'dis_b1' : tf.get_variable('dis_b1', shape=[hidden_dis], dtype=tf.float32, 
                             initializer = tf.zeros_initializer()),
    'dis_b_out' : tf.get_variable('dis_b_out', shape=[1], dtype=tf.float32, 
                             initializer = tf.zeros_initializer())
}

In [65]:
#generator
def generator(x, weights, biases):
    layer1 = tf.nn.relu(tf.matmul(x, weights['gen_w1']) + biases['gen_b1'])
    output = tf.nn.sigmoid(tf.matmul(layer1, weights['gen_w_out']) +  biases['gen_b_out'])
    return output

def discriminator(x, weights, biases):
    layer1 = tf.nn.relu(tf.matmul(x, weights['dis_w1']) + biases['dis_b1'])
    output = tf.nn.sigmoid(tf.matmul(layer1, weights['dis_w_out']) +  biases['dis_b_out'])
    return output



# Build Networks
# Network Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
dis_input = tf.placeholder(tf.float32, shape=[None, image_dim], name='dis_input')

# Build Generator Network
gen_sample = generator(gen_input, weights, biases)

# Build 2 Discriminator Networks (one from noise input, one from generated samples)
dis_real = discriminator(dis_input, weights, biases)
dis_fake = discriminator(gen_sample, weights, biases)

# Build Loss
gen_loss = -tf.reduce_mean(tf.log(dis_fake))
dis_loss = -tf.reduce_mean(tf.log(dis_real) + tf.log(1. - dis_fake))

# Build Optimizers
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_dis = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Training Variables for each optimizer
# By default in TensorFlow, all variables are updated by each optimizer, so we
# need to precise for each one of them the specific variables to update.
# Generator Network Variables
gen_vars = [weights['gen_w1'], weights['gen_w_out'],
            biases['gen_b1'], biases['gen_b_out']]
# Discriminator Network Variables
dis_vars = [weights['dis_w1'], weights['dis_w_out'],
            biases['dis_b1'], biases['dis_b_out']]

# Create training operations
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_dis = optimizer_disc.minimize(dis_loss, var_list=dis_vars)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [66]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)

# Training
for i in range(1, num_steps+1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x, _ = mnist.train.next_batch(batch_size)
    # Generate noise to feed to the generator
    z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])

    # Train
    feed_dict = {dis_input: batch_x, gen_input: z}
    _, _, gl, dl = sess.run([train_gen, train_dis, gen_loss, dis_loss],
                            feed_dict=feed_dict)
    if i % 2000 == 0 or i == 1:
        print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))


Step 1: Generator Loss: 0.432765, Discriminator Loss: 1.652939
Step 2000: Generator Loss: 4.691989, Discriminator Loss: 0.040894
Step 4000: Generator Loss: 4.134645, Discriminator Loss: 0.073377
Step 6000: Generator Loss: 3.682271, Discriminator Loss: 0.088773
Step 8000: Generator Loss: 3.971675, Discriminator Loss: 0.174340
Step 10000: Generator Loss: 3.395542, Discriminator Loss: 0.290145
Step 12000: Generator Loss: 3.707896, Discriminator Loss: 0.172865
Step 14000: Generator Loss: 3.817673, Discriminator Loss: 0.304770
Step 16000: Generator Loss: 3.993147, Discriminator Loss: 0.177676
Step 18000: Generator Loss: 2.653086, Discriminator Loss: 0.387295
Step 20000: Generator Loss: 3.337434, Discriminator Loss: 0.285236
Step 22000: Generator Loss: 3.871713, Discriminator Loss: 0.243435
Step 24000: Generator Loss: 3.855072, Discriminator Loss: 0.315047
Step 26000: Generator Loss: 3.496747, Discriminator Loss: 0.427379
Step 28000: Generator Loss: 3.756812, Discriminator Loss: 0.249019
Ste

In [ ]:
n = 6
canvas = np.empty((28*n, 28*n))

for i in range(n):
    z = np.random.uniform(-1., 1, [n, noise_dim])
    
    g = sess.run(gen_sample, feed_dict={gen_input: z})
    
    g = -1 * (g-1)
    
    for j in range(n):
        canvas[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

plt.figure(figsize=(n, n))
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()